## Prepare Environment

Birinchi navbatda bizga yetarli darajada kuchli bo'lgan GPU kerak bo'ladi va biz Kagglening gpusidan foydalanamiz


Bizdagi GPU ishlayotganini tekshiramiz

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Dec 10 22:02:11 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Biz Whisper modelini fine-tune qilish uchun bir nechta mashhur Python paketlaridan foydalanamiz. datasets[audio] yordamida trening ma'lumotlarini yuklab olib tayyorlaymiz, transformers va accelerate paketlari yordamida modelni yuklab olib o'rgatamiz. Shuningdek, audio fayllarni oldindan ishlov berish uchun soundfile, modelning samaradorligini baholash uchun evaluate va jiwer, metrikalarni yozib borish uchun esa tensorboard paketlaridan foydalanamiz.

In [3]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.1 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.18.0 which is incompatible.


Bizdan Huggingfacega kirish talab qilinadi

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

Birinchi muhim qadam sifatida biz kerakli datasetni yuklab olishimiz kerak bo'ladi, Biz  mozilla-foundation/common_voice_17_0 datasetini ishlatamiz (O'zbek tilida)

Dataset ancha katta, shuning uchun biz uning bir qismidan foydalanamiz

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "uz", split="validated[:25%]")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "uz", split="validated[25%:27%]")

print(common_voice)

README.md:   0%|          | 0.00/12.7k [00:00<?, ?B/s]

common_voice_17_0.py:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/132k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


...

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13936it [00:00, 124114.71it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 15912it [00:00, 159107.63it/s]
Reading metadata...: 31823it [00:00, 157876.09it/s]
Reading metadata...: 47655it [00:00, 158071.83it/s]
Reading metadata...: 63463it [00:00, 157077.80it/s]
Reading metadata...: 86430it [00:00, 148821.99it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 21608
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1728
    })
})


Biz o'zimizga kerakli bo'lgan ma'lumotlarnigina qoldiramiz

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes",'variant'])

print(common_voice)

## Prepare Feature Extractor, Tokenizer and Data


Jarayoni uch bosqichga bo'linadi:

- Xom audio ma'lumotlarni oldindan ishlovchi **feature extractor**.
- Ma'lumotni ketma-ketlikka moslovchi **model**.
- Model chiqishlarini matn formatiga o'tkazuvchi **tokenizer**.

Transformers kutubxonasida Whisper modeli uchun maxsus WhisperFeatureExtractor va WhisperTokenizer mavjud.

### Load WhisperFeatureExtractor

In [9]:
import transformers
transformers.utils.move_cache()

0it [00:00, ?it/s]

In [10]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

### Load WhisperTokenizer

In [11]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="uzbek", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

### Combine To Create A WhisperProcessor

In [12]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="uzbek", task="transcribe")

### Prepare Data

Navbat ma'lumotlarni tozalash qismiga, keling avval datasetdan bir ma'lumotni ko'raylik

In [13]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c65930f7f5ecf6f5e26df4ef01a9be39f15053e7c5c1b4f9e4a0eebb288d5d55/uz_validated_0/common_voice_uz_30061653.mp3', 'array': array([ 2.04281037e-14,  1.49213975e-13,  1.10134124e-13, ...,
       -2.92761655e-07, -2.13421458e-07, -9.10449955e-08]), 'sampling_rate': 48000}, 'sentence': 'Xola, anuv shokoladingizdan obering, dedi chamasi etti yasharlar qiz bidirlab'}


`cast_column` metodida foydalanamiz va audioning `sampling_rate`ini o'zgaritishimiz kerak, chunki whisper modeli 16 kHz ma'lumot uchun ishlay oladi

In [14]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Keling endi shu ma'lumotni qayta ko'ramiz

In [15]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/c65930f7f5ecf6f5e26df4ef01a9be39f15053e7c5c1b4f9e4a0eebb288d5d55/uz_validated_0/common_voice_uz_30061653.mp3', 'array': array([-1.13686838e-13, -2.84217094e-13,  1.13686838e-12, ...,
       -2.05199058e-08, -1.63471810e-07, -2.74593589e-07]), 'sampling_rate': 16000}, 'sentence': 'Xola, anuv shokoladingizdan obering, dedi chamasi etti yasharlar qiz bidirlab'}


Model uchun ma'lumot tayyorlash:

- Audio yuklanadi va qayta namuna olinadi.
- Log-Mel spektrogram xususiyatlari olinadi.
- Transkripsiyalar label IDlarga kodlanadi.







In [16]:
def prepare_dataset(batch):

    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch

**Bizda `num_proc` parametri muhim parametr hisoblanadi ya'ni bu bizga multi-processing imkonini beradi. Uni to'g'ri tanlash uchun biz cpu yadrolari soni aniqlashimiz kerak va shunga teng bo'lgan qiymat tanlaymiz**

In [17]:
import os
print(os.cpu_count())

4


`.map` metodi orqali ma'lumotni tayyorlaymiz.

Jarayon biroz vaqt oladi

In [18]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/21608 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1728 [00:00<?, ? examples/s]

## Training and Evaluation

Endi ma'lumot tayyorlandi, trening jarayoniga o'tamiz.
Qadamlar:

- **Pre-trained checkpoint yuklash**: modelni to'g'ri o'rnatish va treningga tayyorlash.
- **Data collator aniqlash**: oldindan ishlov berilgan ma'lumotlarni PyTorch tensorga tayyorlash.
- **Baholash metrikalari**: WER (word error rate) metrikasidan foydalanib, modelni baholash.
- **Treaning konfiguratsiyasini belgilash**: Trainer uchun trening jadvalini aniqlash.
Fine-tune qilingan modelni sinov ma'lumotlari ustida baholash

# Load a Pre-Trained Checkpoint

Fine-tune jarayonini pre-trained `Whisper medium` checkpointdan boshlaymiz. Bu checkpointning wieghtlarini Hugging Face Hubdan yuklab olishimiz kerak.

In [19]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("jamshidahmadov/whisper-uz")

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

Inference paytida modelni faqat O'zbek tilida ishlashga moslashtirish uchun automatic speech recognationni o'chiramiz.



---


**Inference — bu modelni o'qitishdan so'ng, yangi ma'lumotlar bilan test qilish yoki undan natija olish jarayonidir. Masalan, modelni treningdan so'ng, unga audio yoki matn kiritib, modeldan javob olish (masalan, nutqni matnga aylantirish) inference deb ataladi.**

In [20]:
model.generation_config.language = "uzbek"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

Data collator `input_features` va `labels`ni alohida ishlaydi. `input_features`ni `feature extractor` bilan, `labels`ni esa `tokenizer` bilan qayta ishlaymiz.

In [21]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [22]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

Modelni baholash uchun standart bo'lib hisoblangan `word error rate (WER)`dan foydalanamiz

In [23]:
import evaluate

metric = evaluate.load("wer")

`compute_metrics` funksiyasi -100ni `pad_token_id`ga almashtirib, `predict` va `label` IDlarini matnga decoding qiladi va WERni hisoblaydi.

In [24]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

Oxirgi qadam: Treaning uchun kerakli parametrlarni yozamiz

In [30]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-uz-v2",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=3000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Treaning argumentlarini trainerga, `model`, `dataset`, `data collator` va `compute_metrics` funksiyasi bilan birga uzatamiz:

In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/tmp/ipykernel_23/240849381.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [32]:
import torch
torch.cuda.empty_cache() 

Treningni boshlashdan oldin, processor obyektini saqlab olish maslahat beriladi. U training jarayonida o'zgarmaydi

In [33]:
processor.save_pretrained(training_args.output_dir)

[]

### Training

**Training jarayoniga ham yetib keldik. Modelni train qilamiz...**

In [ ]:
trainer.train()

Wer ~ 65 natijasiga ega bo'ldik.
Ko'proq resurslar va datasetning kattaroq qismidan foydalanish orqali bu natijani oshira olamiz

Endi modelni huggingfacega yuklaymiz

In [35]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_17_0",
    "dataset": "Common Voice 17.0", 
    "dataset_args": "config: uz, split: test",
    "language": "uz",
    "model_name": "Whisper base uz V2 - Jamshid Ahmadov", 
    "finetuned_from": "jamshidahmadov/whisper-uz",
    "tasks": "automatic-speech-recognition",
}

 `push_to_hub` buyrug'idan foydalanamiz:

In [37]:
trainer.push_to_hub(**kwargs)

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1733869048.83e89f1d4b72.23.0:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

events.out.tfevents.1733869432.83e89f1d4b72.23.1:   0%|          | 0.00/32.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jamshidahmadov/whisper-uz-v2/commit/34f499c8889994e76b1bf8fc882dde283bc237c0', commit_message='End of training', commit_description='', oid='34f499c8889994e76b1bf8fc882dde283bc237c0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jamshidahmadov/whisper-uz-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='jamshidahmadov/whisper-uz-v2'), pr_revision=None, pr_num=None)

STT Model tayyor